In [1]:
%run block_swipdg.ipynb

initializing grid and problem ...
preparing sparsity patterns ...
assembling locally ...
assembling globally ...
solving ...


In [3]:
import numpy as np

from dune.gdt import (
    apply_oswald_interpolation_operator,
    RS2017_make_elliptic_matrix_operator_on_subdomain
        as make_local_elliptic_matrix_operator,
    make_ESV2007_nonconformity_product_dd_subdomain_part_dd_subdomain_oversampled_part
        as make_local_nonconformity_product,
    make_rt_leaf_view_to_2x1_pdelab_p0_space as make_rt_space,
    RS2017_apply_diffusive_flux_reconstruction_in_neighborhood
        as apply_diffusive_flux_reconstruction_in_neighborhood,
    RS2017_apply_l2_product as apply_l2_product,
    RS2017_residual_indicator_min_diffusion_eigenvalue as min_diffusion_eigenvalue,
    RS2017_residual_indicator_subdomain_diameter as subdomain_diameter,
    RS2017_diffusive_flux_indicator_apply_aa_product as apply_diffusive_flux_aa_product,
    RS2017_diffusive_flux_indicator_apply_ab_product as apply_diffusive_flux_ab_product,
    RS2017_diffusive_flux_indicator_apply_bb_product as apply_diffusive_flux_bb_product,
    RS2017_make_penalty_product_matrix_operator_on_oversampled_subdomain
        as make_penalty_product_matrix_operator
)

print('estimating error ', end='')

# fake parametric situation
lambda_bar = lambda_
lambda_hat = lambda_
lambda_xi = lambda_
lambda_xi_prime = lambda_
alpha_mu_mu_bar = 1.
gamma_mu_mu_bar = 1.
alpha_mu_mu_hat = 1.

# localized_u_h contains the vector of the solution restricted to each subdomain
local_sizes = [block_space.local_space(ii).size()
               for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii]))
                for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]]
                    for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(block_space.local_space(ii).size(), 0.)
                 for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]


def localize(neighborhood_vector, block_space, neighborhood):
    neighborhood = list(neighborhood)
    _local_sizes = [block_space.local_space(nn).size() for nn in neighborhood]
    _local_starts = [int(np.sum(_local_sizes[:ii])) for ii in range(len(neighborhood))]
    _local_starts.append(len(neighborhood_vector))
    _localized_u_h_np = np.array(neighborhood_vector, copy=False)
    _localized_u_h_np = [_localized_u_h_np[_local_starts[ii]:_local_starts[ii+1]]
                        for ii in range(len(neighborhood))]
    _localized_u_h = {ii: Vector(block_space.local_space(ii).size(), 0.)
                     for ii in neighborhood}
    for ii in range(len(neighborhood)):
        nn = neighborhood[ii]
        tmp = np.array(_localized_u_h[nn], copy=False)
        tmp[:] = _localized_u_h_np[ii][:]
    return _localized_u_h


local_boundary_info = make_subdomain_boundary_info(
    grid,
    {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
     'default': 'dirichlet',
     'neumann': '[{} {}]'.format(inner_boundary_id, inner_boundary_id+1)})

global_rt_space = make_rt_space(grid)

local_eta_nc_squared = [0. for ii in range(grid.num_subdomains)]
local_eta_r_squared = [0. for ii in range(grid.num_subdomains)]
local_eta_df_squared = [0. for ii in range(grid.num_subdomains)]
penalty_products = [0. for ii in range(grid.num_subdomains)]

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)

    penalty_product = make_penalty_product_matrix_operator(grid,
                                                           ii,
                                                           local_boundary_info,
                                                           neighborhood_space,
                                                           lambda_bar,
                                                           kappa,
                                                           over_integrate=0)
    penalty_product.assemble()
    
    elliptic_product = make_local_elliptic_matrix_operator(grid, ii, block_space.local_space(ii),
                                                           lambda_hat, kappa)
    elliptic_product.assemble()
    elliptic_product = elliptic_product.matrix()

    subdomain_uh = make_discrete_function(block_space.local_space(ii), localized_u_h[ii])

    def localize_to_subdomain_with_neighborhood_support(ss):
        return make_discrete_function(
            neighborhood_space,
            neighborhood_space.project_onto_neighborhood(
                [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                 for nn in neighborhood],
                neighborhood))
    subdomain_uh_with_neighborhood_support = {nn: localize_to_subdomain_with_neighborhood_support(nn)
                                              for nn in neighborhood}

    # The following is not optimal, but the pdelab-based RT space does not work on the
    # dd_subdomain_oversampled grid view modeling the neighborhood (at least is the
    # underlying grid is a 2d simplex alugrid) and we thus need to reconstruct on the
    # full leaf view.
    def localize_to_subdomain_with_global_support(ss):
        return make_discrete_function(
            block_space,
            block_space.project_onto_neighborhood(
                [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                 for nn in range(grid.num_subdomains)],
                set([nn for nn in range(grid.num_subdomains)])))
    subdomain_uhs_with_global_support = {nn: localize_to_subdomain_with_global_support(nn)
                                         for nn in neighborhood}

    # eta r, f x f part
    local_eta_r_squared[ii] += apply_l2_product(grid, ii, f, f, over_integrate=0)
    # eta df, u x u part
    local_eta_df_squared[ii] += apply_diffusive_flux_aa_product(
        grid, ii,
        lambda_hat, lambda_u=lambda_xi, lambda_v=lambda_xi_prime,
        kappa=kappa,
        u=subdomain_uh,
        v=subdomain_uh,
        over_integrate=0)

    for jj in neighborhood:
        reconstructed_uh_jj_with_global_support = make_discrete_function(global_rt_space)
        apply_diffusive_flux_reconstruction_in_neighborhood(
            grid, ii,
            lambda_xi, kappa,
            subdomain_uhs_with_global_support[jj],
            reconstructed_uh_jj_with_global_support,
            over_integrate=0)
        # eta r, f x reconstructed_u part
        local_eta_r_squared[ii] -= 2.*apply_l2_product(
            grid, ii,
            f,
            reconstructed_uh_jj_with_global_support.divergence(),
            over_integrate=0)
        # eta df, u x reconstructed_u part
        local_eta_df_squared[ii] += 2.*apply_diffusive_flux_ab_product(
            grid, ii,
            lambda_hat,
            lambda_u=lambda_xi_prime,
            kappa=kappa,
            u=subdomain_uh,
            reconstructed_v=reconstructed_uh_jj_with_global_support,
            over_integrate=0)
        
        for kk in neighborhood:
            reconstructed_uh_kk_with_global_support = make_discrete_function(global_rt_space)
            apply_diffusive_flux_reconstruction_in_neighborhood(
                grid, ii,
                lambda_xi_prime, kappa,
                subdomain_uhs_with_global_support[kk],
                reconstructed_uh_kk_with_global_support,
                over_integrate=0)
            # eta nc
            foo = make_local_nonconformity_product(
                grid, ii, ii, local_boundary_info,
                lambda_hat, kappa,
                subdomain_uh_with_neighborhood_support[jj],
                subdomain_uh_with_neighborhood_support[kk],
                over_integrate=0).apply2()
            #print('{}, {}, {}: {}'.format(ii, jj, kk, foo))
            interpolated_u_vector = neighborhood_space.project_onto_neighborhood(
                [Vector(block_space.local_space(nn).size(), 0.) for nn in neighborhood], neighborhood)
            interpolated_u = make_discrete_function(neighborhood_space, interpolated_u_vector)
            interpolated_v_vector = neighborhood_space.project_onto_neighborhood(
                [Vector(block_space.local_space(nn).size(), 0.) for nn in neighborhood], neighborhood)
            interpolated_v = make_discrete_function(neighborhood_space, interpolated_v_vector)
            apply_oswald_interpolation_operator(grid, ii,
                                                make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.alldirichlet'}),
                                                subdomain_uh_with_neighborhood_support[jj],
                                                interpolated_u)
            apply_oswald_interpolation_operator(grid, ii,
                                                make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.alldirichlet'}),
                                                subdomain_uh_with_neighborhood_support[kk],
                                                interpolated_v)
            u_diff = (
                localize(subdomain_uh_with_neighborhood_support[jj].vector_copy(), block_space, neighborhood)[ii]
                - localize(interpolated_u_vector, block_space, neighborhood)[ii])
            v_diff = (
                localize(subdomain_uh_with_neighborhood_support[kk].vector_copy(), block_space, neighborhood)[ii]
                - localize(interpolated_v_vector, block_space, neighborhood)[ii])            
            bar = (elliptic_product * v_diff) * u_diff
            #print('{}, {}, {}: {}'.format(ii, jj, kk, bar))
            local_eta_nc_squared[ii] += bar
            # eta r, reconstructed_u x reconstructed_u part
            local_eta_r_squared[ii] += apply_l2_product(
                grid, ii,
                reconstructed_uh_jj_with_global_support.divergence(),
                reconstructed_uh_kk_with_global_support.divergence(),
                over_integrate=0)
            # eta df, reconstructed_u x reconstructed_u part
            local_eta_df_squared[ii] += apply_diffusive_flux_bb_product(
                grid, ii,
                lambda_hat, kappa,
                reconstructed_uh_jj_with_global_support,
                reconstructed_uh_kk_with_global_support,
                over_integrate=0)
            # penalty product
            penalty_products[ii] += penalty_product.apply2(
                subdomain_uh_with_neighborhood_support[jj],
                subdomain_uh_with_neighborhood_support[kk])

    # eta r, scale
    poincaree_constant = 1./(np.pi**2)
    min_diffusion_ev = min_diffusion_eigenvalue(grid, ii, lambda_hat, kappa)
    subdomain_h = subdomain_diameter(grid, ii)
    local_eta_r_squared[ii] *= (poincaree_constant/min_diffusion_ev) * subdomain_h**2

    print('.', end='')

print('')
print('  nonconformity indicator:   {} (should be 1.66e-01)'.format(
    np.sqrt(np.sum(local_eta_nc_squared))))
print('  residual indicator:        {} (should be 2.89e-01)'.format(
    np.sqrt(np.sum(local_eta_r_squared))))
print('  diffusive flux indicator:  {} (should be 3.55e-01)'.format(
    np.sqrt(np.sum(local_eta_df_squared))))

eta_tilde = 0.
eta_tilde +=     np.sqrt(gamma_mu_mu_bar)  * np.sqrt(np.sum(local_eta_nc_squared))
eta_tilde += (1./np.sqrt(alpha_mu_mu_hat)) * np.sqrt(np.sum((  np.sqrt(local_eta_r_squared)
                                                             + np.sqrt(local_eta_df_squared))**2))
eta_tilde *=  1./np.sqrt(alpha_mu_mu_bar)

print('  estimated semi-norm error: {} (should be 0.8099...)'.format(eta_tilde))

print('')
print('  penalty product:                {} (should be 0.2210...)'.format(np.sum(penalty_products)))
eta = np.sqrt(eta_tilde**2 + np.sum(penalty_products))
print('  estimated energy-DG norm error: {} (should be 0.9365...)'.format(eta))

estimating error 

TypeError: project_onto_neighborhood(): incompatible function arguments. The following argument types are supported:
    1. (self: dune.gdt.__spaces_block.BlockDg2dSimplexAluconformgridDdSubdomainPartTo1x1FemP1Space, local_vectors: List[dune.xt.la._la.IstlDenseVectorDouble], neighborhood: List[int]) -> dune.xt.la._la.IstlDenseVectorDouble

Invoked with: <dune.gdt.__spaces_block.BlockDg2dSimplexAluconformgridDdSubdomainPartTo1x1FemP1Space object at 0x7f9244261ae0>, [IstlDenseVectorDouble([0.145463 0.11147 0.0832944 ... 0.0832944 0.11147 0.145463]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0 0 ... 0 0 0]), IstlDenseVectorDouble([0 0